In [1]:
import fastf1
import requests
from collections import defaultdict
from pprint import pprint

SEASON = 2023
FIA_POINTS = {1: 25, 2: 18, 3: 15, 4: 12, 5: 10, 6: 8, 7: 6, 8: 4, 9: 2, 10: 1}

# Cache do FastF1 (para não baixar sempre)
fastf1.Cache.enable_cache(".fastf1_cache")


In [34]:
# 1) OpenF1: metadados de pilotos (cores, headshot, país)
openf1_resp = requests.get("https://api.openf1.org/v1/drivers")#, params={"limit": 5000})
openf1_resp.raise_for_status()
openf1_data = openf1_resp.json()

In [35]:
openf1_data

[{'meeting_key': 1140,
  'session_key': 7763,
  'driver_number': 1,
  'broadcast_name': 'M VERSTAPPEN',
  'full_name': 'Max VERSTAPPEN',
  'name_acronym': 'VER',
  'team_name': 'Red Bull Racing',
  'team_colour': '3671C6',
  'first_name': 'Max',
  'last_name': 'Verstappen',
  'headshot_url': 'https://www.formula1.com/content/dam/fom-website/drivers/M/MAXVER01_Max_Verstappen/maxver01.png.transform/1col/image.png',
  'country_code': 'NED'},
 {'meeting_key': 1140,
  'session_key': 7763,
  'driver_number': 2,
  'broadcast_name': 'L SARGEANT',
  'full_name': 'Logan SARGEANT',
  'name_acronym': 'SAR',
  'team_name': 'Williams',
  'team_colour': '37BEDD',
  'first_name': 'Logan',
  'last_name': 'Sargeant',
  'headshot_url': 'https://www.formula1.com/content/dam/fom-website/drivers/L/LOGSAR01_Logan_Sargeant/logsar01.png.transform/1col/image.png',
  'country_code': 'USA'},
 {'meeting_key': 1140,
  'session_key': 7763,
  'driver_number': 4,
  'broadcast_name': 'L NORRIS',
  'full_name': 'Lando N

In [ ]:
# 1) OpenF1: metadados de pilotos (cores, headshot, país)
openf1_resp = requests.get("https://api.openf1.org/v1/drivers")#, params={"limit": 5000})
openf1_resp.raise_for_status()
openf1_data = openf1_resp.json()

# dicionário por driver_number com a entrada mais recente (maior session_key)
openf1_lookup = {}
for entry in openf1_data:
    num = entry.get("driver_number")
    if num is None:
        continue
    num = str(num)
    if num not in openf1_lookup or entry.get("session_key", 0) > openf1_lookup[num].get("session_key", 0):
        openf1_lookup[num] = entry

print(f"Pilotos encontrados no OpenF1: {len(openf1_lookup)}")
pprint(list(openf1_lookup.items())[:2])


In [10]:
# 2) FastF1: calendário da temporada
schedule = fastf1.get_event_schedule(SEASON, include_testing=False)
print(schedule[["RoundNumber", "EventName"]].head())


   RoundNumber                 EventName
1            1        Bahrain Grand Prix
2            2  Saudi Arabian Grand Prix
3            3     Australian Grand Prix
4            4     Azerbaijan Grand Prix
5            5          Miami Grand Prix


In [ ]:
# 3) FastF1: tentar carregar resultados da primeira etapa
round_number = int(schedule.iloc[0]["RoundNumber"])
session = fastf1.get_session(SEASON, round_number, "R")
session.load(laps=False, telemetry=False)

classification = session.results
print("Resultados FastF1/Ergast?", classification is not None and len(classification) > 0)

if classification is not None and len(classification) > 0:
    print(classification.head())
else:
    print("Sem resultados via FastF1 nesta sessão.")


core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.3.5]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
logger      WARNING 	Failed to load result data from Ergast!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '16', '55', '14', '63', '44', '18', '31', '27', '4', '77', '24', '22', '23', '2', '20', '81', '21', '10']


Resultados FastF1/Ergast? True
   DriverNumber BroadcastName Abbreviation DriverId         TeamName  \
1             1  M VERSTAPPEN          VER           Red Bull Racing   
11           11       S PEREZ          PER           Red Bull Racing   
16           16     C LECLERC          LEC                   Ferrari   
55           55       C SAINZ          SAI                   Ferrari   
14           14      F ALONSO          ALO              Aston Martin   

   TeamColor TeamId FirstName    LastName         FullName  ... CountryCode  \
1     3671C6              Max  Verstappen   Max Verstappen  ...         NED   
11    3671C6           Sergio       Perez     Sergio Perez  ...         MEX   
16    F91536          Charles     Leclerc  Charles Leclerc  ...         MON   
55    F91536           Carlos       Sainz     Carlos Sainz  ...         ESP   
14    358C75         Fernando      Alonso  Fernando Alonso  ...         ESP   

   Position  ClassifiedPosition GridPosition  Q1  Q2  Q3 Time

In [17]:
# 4) Ergast direto (fallback)
ergast_url = f"http://ergast.com/api/f1/{SEASON}/results.json"
ergast_resp = requests.get(ergast_url)
print("Status Ergast:", ergast_resp.status_code)

ergast_data = ergast_resp.json() if ergast_resp.ok else {}
races = ergast_data.get("MRData", {}).get("RaceTable", {}).get("Races", [])
print(f"Corridas retornadas pelo Ergast: {len(races)}")

if races:
    first_race = races[0]
    print("Exemplo de resultado Ergast (primeira corrida, 3 primeiros):")
    pprint(first_race.get("Results", [])[:3])


Status Ergast: 403
Corridas retornadas pelo Ergast: 0


In [20]:
ergast_url = f"http://ergast.com/api/f1/1957/results.json"
ergast_resp = requests.get(ergast_url)

In [21]:
print("Status Ergast:", ergast_resp.status_code)


Status Ergast: 403


In [24]:
ergast_resp.status_code

403

In [29]:
import requests

url = "http://ergast.com/mrd/1.5/f1/1950/results.json"
params = {"limit": 500}
headers = {
    "User-Agent": "Mozilla/5.0",
    "Accept": "application/json",
}

resp = requests.get(url, params=params, headers=headers, timeout=30)
print("Status:", resp.status_code)
print("Content-Type:", resp.headers.get("Content-Type"))
resp.raise_for_status()

data = resp.json()
print("OK — corridas:", len(data["MRData"]["RaceTable"]["Races"]))


Status: 441
Content-Type: None


HTTPError: 441 Client Error:  for url: http://ergast.com/mrd/1.5/f1/1950/results.json?limit=500

In [32]:
schedule = fastf1.get_event_schedule(2024, include_testing=False)

In [ ]:
resp = client.get(f"{OPENF1_BASE_URL}/drivers")
resp.raise_for_status()
data = resp.json()

,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session1DateUtc,...,Session3,Session3Date,Session3DateUtc,Session4,Session4Date,Session4DateUtc,Session5,Session5Date,Session5DateUtc,F1ApiSupport
1,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,2024-03-02,Bahrain Grand Prix,conventional,Practice 1,2024-02-29 14:30:00+03:00,2024-02-29 11:30:00,...,Practice 3,2024-03-01 15:30:00+03:00,2024-03-01 12:30:00,Qualifying,2024-03-01 19:00:00+03:00,2024-03-01 16:00:00,Race,2024-03-02 18:00:00+03:00,2024-03-02 15:00:00,True
2,2,Saudi Arabia,Jeddah,FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2024,2024-03-09,Saudi Arabian Grand Prix,conventional,Practice 1,2024-03-07 16:30:00+03:00,2024-03-07 13:30:00,...,Practice 3,2024-03-08 16:30:00+03:00,2024-03-08 13:30:00,Qualifying,2024-03-08 20:00:00+03:00,2024-03-08 17:00:00,Race,2024-03-09 20:00:00+03:00,2024-03-09 17:00:00,True
3,3,Australia,Melbourne,FORMULA 1 ROLEX AUSTRALIAN GRAND PRIX 2024,2024-03-24,Australian Grand Prix,conventional,Practice 1,2024-03-22 12:30:00+11:00,2024-03-22 01:30:00,...,Practice 3,2024-03-23 12:30:00+11:00,2024-03-23 01:30:00,Qualifying,2024-03-23 16:00:00+11:00,2024-03-23 05:00:00,Race,2024-03-24 15:00:00+11:00,2024-03-24 04:00:00,True
4,4,Japan,Suzuka,FORMULA 1 MSC CRUISES JAPANESE GRAND PRIX 2024,2024-04-07,Japanese Grand Prix,conventional,Practice 1,2024-04-05 11:30:00+09:00,2024-04-05 02:30:00,...,Practice 3,2024-04-06 11:30:00+09:00,2024-04-06 02:30:00,Qualifying,2024-04-06 15:00:00+09:00,2024-04-06 06:00:00,Race,2024-04-07 14:00:00+09:00,2024-04-07 05:00:00,True
5,5,China,Shanghai,FORMULA 1 LENOVO CHINESE GRAND PRIX 2024,2024-04-21,Chinese Grand Prix,sprint_qualifying,Practice 1,2024-04-19 11:30:00+08:00,2024-04-19 03:30:00,...,Sprint,2024-04-20 11:00:00+08:00,2024-04-20 03:00:00,Qualifying,2024-04-20 15:00:00+08:00,2024-04-20 07:00:00,Race,2024-04-21 15:00:00+08:00,2024-04-21 07:00:00,True
6,6,United States,Miami,FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2024,2024-05-05,Miami Grand Prix,sprint_qualifying,Practice 1,2024-05-03 12:30:00-04:00,2024-05-03 16:30:00,...,Sprint,2024-05-04 12:00:00-04:00,2024-05-04 16:00:00,Qualifying,2024-05-04 16:00:00-04:00,2024-05-04 20:00:00,Race,2024-05-05 16:00:00-04:00,2024-05-05 20:00:00,True
7,7,Italy,Imola,FORMULA 1 MSC CRUISES GRAN PREMIO DEL MADE IN ...,2024-05-19,Emilia Romagna Grand Prix,conventional,Practice 1,2024-05-17 13:30:00+02:00,2024-05-17 11:30:00,...,Practice 3,2024-05-18 12:30:00+02:00,2024-05-18 10:30:00,Qualifying,2024-05-18 16:00:00+02:00,2024-05-18 14:00:00,Race,2024-05-19 15:00:00+02:00,2024-05-19 13:00:00,True
8,8,Monaco,Monaco,FORMULA 1 GRAND PRIX DE MONACO 2024,2024-05-26,Monaco Grand Prix,conventional,Practice 1,2024-05-24 13:30:00+02:00,2024-05-24 11:30:00,...,Practice 3,2024-05-25 12:30:00+02:00,2024-05-25 10:30:00,Qualifying,2024-05-25 16:00:00+02:00,2024-05-25 14:00:00,Race,2024-05-26 15:00:00+02:00,2024-05-26 13:00:00,True
9,9,Canada,Montréal,FORMULA 1 AWS GRAND PRIX DU CANADA 2024,2024-06-09,Canadian Grand Prix,conventional,Practice 1,2024-06-07 13:30:00-04:00,2024-06-07 17:30:00,...,Practice 3,2024-06-08 12:30:00-04:00,2024-06-08 16:30:00,Qualifying,2024-06-08 16:00:00-04:00,2024-06-08 20:00:00,Race,2024-06-09 14:00:00-04:00,2024-06-09 18:00:00,True
10,10,Spain,Barcelona,FORMULA 1 ARAMCO GRAN PREMIO DE ESPAÑA 2024,2024-06-23,Spanish Grand Prix,conventional,Practice 1,2024-06-21 13:30:00+02:00,2024-06-21 11:30:00,...,Practice 3,2024-06-22 12:30:00+02:00,2024-06-22 10:30:00,Qualifying,2024-06-22 16:00:00+02:00,2024-06-22 14:00:00,Race,2024-06-23 15:00:00+02:00,2024-06-23 13:00:00,True
